In [3]:
#CLSP-Rmin with back-ordering example
from gurobipy import *
import time
import math
import random
from function_call import compute_values
from function_call_clsp import compute_values as cpCLSP 
from data import variables
import json


#start = time.time()

n = Model("CLSP-RminBO")

#Parameters
d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe = variables()

# generate random ratio for defective items
random.seed(123)
o1 = round(random.uniform(0.01,0.02), 3)# realistic
o2 = round(random.uniform(0.02,0.03), 3) # pessimistic 
o3 = round(random.uniform(0.001,0.01), 3) # optimistic 
O=[o1 for t in Pe] # realistic
O1=[o2 for t in Pe ]  # pessimistic
O2=[o3 for t in Pe] # optimistic 

lista=[{'O':O, 'O1':O1,'O2':O2}]
with open('data.json', 'w') as fa:
    json.dump(lista,fa)
json_str = json.dumps(lista)
print(json_str)
with open('data.json', 'r') as fa:
    thisfile=json.load(fa)
O=thisfile[0]["O"]


#################################### Run optymalization ############################# 


Pe=range(T)    
CostBO=True
Rp=10
cpCLSP(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp,CostBO)


T=6    
Pe=range(T)    
Rp=2

for i in range(5):  
    if i ==0:
        print('step',i+1)
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp)
    
    elif i==3: #and T >len(b) and Rp<len(b):
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        print(BO0)
        print(s0)
        print(sR0)
        print(b)
        print(d)
        Pe=range(len(b))
        T=len(b)
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp)
    
    elif i==4: #T>len(b) and Rp>len(b)
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        Pe=range(len(b))
        T=len(b)
        Rp=len(b)
        print(BO0)
        print(s0)
        print(sR0)
        print(b)
        print(d)
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp)
    
    elif i<3:
        print('step',i+1)
        for j in d:
            for q in range(Rp):
                del j[0]
        for z in range(Rp):
            b.pop(0)
        print(i)
        print(len(b))
        print(b)
        [BO0,s0,sR0] = compute_values(n,d,f,h,hR,tp,tpR,ts,tsR,b,k,M,s0,sR0,T,m,BO0,Pro,Pe,O,Rp)

    else:
        print('end')


[{"O": [0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011, 0.011], "O1": [0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021, 0.021], "O2": [0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005]}]
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 582 rows, 360 columns and 1440 nonzeros
Model fingerprint: 0xfb09c4f0
Variable types: 120 continuous, 240 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+03]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 256 rows and 16 columns
Presolve time: 0.03s
Presolved: 326 rows, 344 columns, 1096 nonzeros
Variable types: 72 continuous, 272 integer (80 binary)

Root relaxation: objective 7.086117e+03, 271 iterations, 0.00 seconds 

     yR[0,8]           33 
     yR[1,3]           15 
     yR[1,9]           31 
     yR[2,3]           22 
     yR[2,9]           46 
     yR[3,3]           30 
     yR[3,9]           35 

    Variable            x 
-------------------------
     sR[0,0]            3 
     sR[0,1]            6 
     sR[0,2]           11 
     sR[0,3]           11 
     sR[0,4]           17 
     sR[0,5]           17 
     sR[0,6]           26 
     sR[0,7]           26 
     sR[1,0]            4 
     sR[1,1]            8 
     sR[1,2]           11 
     sR[1,4]            4 
     sR[1,5]            9 
     sR[1,6]           15 
     sR[1,7]           20 
     sR[1,8]           26 
     sR[2,0]            6 
     sR[2,1]           12 
     sR[2,2]           17 
     sR[2,4]            6 
     sR[2,5]           13 
     sR[2,6]           21 
     sR[2,7]           29 
     sR[2,8]           38 
     sR[3,0]            6 
     sR[3,1]           12 
     sR[3,2]           17 
     sR[3,4]            7 
 

     sR[2,0]            6 
     sR[2,1]           12 
     sR[2,3]            5 
     sR[2,4]           11 
     sR[3,0]            6 
     sR[3,1]           12 
     sR[3,2]           17 
     sR[3,3]           22 
     sR[3,4]           28 
0.0 Table 4 h Holding cost
372.75 Table 4 hR Holding cost for rework
3250.0 Setup cost combined
2700.0 Table 4 f Setup cost for production
24.0 Table 4 x Nr of set-ups for production
550.0 Table 4 f Setup cost for rework
5.0 Table 4 xR Setup cost for rework
0.47891249999999996 Table 4 Capacity Utilization
3622.75 Table 4 Z objective function value
0.0 Table 4 Back order costs cb
step 2
1
8
[2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000]
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1282 rows, 792 columns and 3160 nonzeros
Model fingerprint: 0x365d015c
Va

     sR[2,2]           17 
     sR[2,4]            6 
     sR[2,5]           13 
     sR[2,6]           21 
     sR[2,7]           29 
     sR[2,8]           38 
     sR[3,0]            6 
     sR[3,1]           12 
     sR[3,2]           17 
     sR[3,4]            7 
     sR[3,5]           18 
     sR[3,6]           18 
     sR[3,7]           27 
     sR[3,8]           27 
     sR[0,0]            9 
     sR[0,1]           12 
     sR[0,2]           18 
     sR[0,3]           18 
     sR[1,0]           11 
     sR[1,1]           15 
     sR[1,2]           19 
     sR[1,3]           24 
     sR[1,4]           29 
     sR[2,0]           17 
     sR[2,2]            6 
     sR[2,3]           13 
     sR[2,4]           21 
     sR[3,0]           17 
     sR[3,1]           22 
     sR[3,2]           29 
     sR[3,3]           40 
     sR[3,4]           42 
1878.250000000002 Table 4 h Holding cost
611.5 Table 4 hR Holding cost for rework
3030.0 Setup cost combined
2480.0 Table 4 f Setup cost

     sR[1,5]            9 
     sR[1,6]           15 
     sR[1,7]           20 
     sR[1,8]           26 
     sR[2,0]            6 
     sR[2,1]           12 
     sR[2,2]           17 
     sR[2,4]            6 
     sR[2,5]           13 
     sR[2,6]           21 
     sR[2,7]           29 
     sR[2,8]           38 
     sR[3,0]            6 
     sR[3,1]           12 
     sR[3,2]           17 
     sR[3,4]            7 
     sR[3,5]           18 
     sR[3,6]           18 
     sR[3,7]           27 
     sR[3,8]           27 
     sR[0,0]            6 
     sR[0,1]            6 
     sR[0,2]            6 
     sR[0,3]            6 
     sR[0,4]            6 
     sR[2,0]           12 
     sR[2,1]           12 
     sR[2,2]           12 
     sR[0,0]           12 
     sR[0,1]           12 
     sR[1,0]           23 
     sR[1,1]           23 
     sR[1,2]           23 
     sR[1,3]           23 
     sR[1,4]           23 
     sR[2,1]           22 
     sR[2,2]           22 
 

-------------------------
      y[0,0]          910 
      y[0,1]           21 
      y[0,2]            4 
      y[0,3]            1 
      y[0,4]            1 
      y[0,5]            1 
      y[0,6]            1 
      y[0,7]            1 
      y[0,8]            1 
      y[0,9]         1910 
      y[1,0]          910 
      y[1,1]         1819 
      y[1,2]         1001 
      y[1,3]           21 
      y[1,4]           21 
      y[1,5]           21 
      y[1,6]            1 
      y[1,7]            1 
      y[1,8]            1 
      y[2,0]            1 
      y[2,1]            1 
      y[2,2]          819 
      y[2,3]         1819 
      y[2,4]         1819 
      y[2,5]         1182 
      y[2,6]            1 
      y[2,7]            1 
      y[2,8]           57 
      y[3,0]            1 
      y[3,1]            1 
      y[3,2]            1 
      y[3,3]            1 
      y[3,4]            1 
      y[3,5]          637 
      y[3,6]         1819 
      y[3,7]         1819 
  

     yR[1,8]            1 
     yR[2,0]            1 
     yR[2,1]            1 
     yR[2,2]           10 
     yR[2,3]           21 
     yR[2,4]           21 
     yR[2,5]           14 
     yR[2,6]            1 
     yR[2,7]            1 
     yR[2,8]            1 
     yR[3,0]            1 
     yR[3,1]            1 
     yR[3,2]            1 
     yR[3,3]            1 
     yR[3,4]            1 
     yR[3,5]            8 
     yR[3,6]           21 
     yR[3,7]           21 
     yR[3,8]           16 
     yR[3,9]            1 
     yR[0,5]            6 
     yR[1,0]            8 
     yR[2,2]           12 
     yR[3,0]           12 
     yR[0,2]           12 
     yR[1,1]           15 
     yR[3,0]           22 
     yR[0,2]           12 
     yR[1,0]           23 
     yR[2,3]           22 
     yR[3,0]           37 
      y[1,0]          880 
      y[1,1]          969 
      y[2,0]         1090 
      y[2,1]          835 
     yS[1,0]          870 
     yS[1,1]          958 
 